# 机器学习调参练习

在机器学习中，超参数是指无法从数据中学习而需要在训练前提供的参数。机器学习模型的性能在很大程度上依赖于寻找最佳超参数集。

超参数调整一般是指调整模型的超参数，这基本上是一个非常耗时的过程。在本文中，我们将和你一起研习 3 种最流行的超参数调整技术：

- **网格搜索**

- **随机搜索**

- **贝叶斯搜索**

其实还有第零种调参方法，就是手动调参，因为简单机械，就不在本文讨论范围内。

为方便阅读，列出本文的结构如下：

1.获取和准备数据 

2.网格搜索 

3.随机搜索 

4.贝叶斯搜索 

5.写在最后

## 获取和准备数据



为演示方便，本文使用内置乳腺癌数据来训练**支持向量分类**（SVC）。可以通过`load_breast_cancer`函数获取数据。

In [1]:
import pandas as pd
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
df_X = pd.DataFrame(cancer['data'], columns=cancer['feature_names'])
df_X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


接下来为特征和目标标签创建`df_X`和`df_y`，如下所示：

In [2]:
df_y = pd.DataFrame(cancer['target'], columns=['Cancer'])
df_y.head()

,Cancer
0,0
1,0
2,0
3,0
4,0


> PS ：如果想了解更多关于数据集的信息，可以运行`print(cancer['DESCR'])`打印出摘要和特征信息。

In [3]:
# print(cancer['DESCR'])

接下来，使用`training_test_split()`方法将数据集拆分为训练集 \(70\%\) 和测试集 \(30\%\) ：

In [4]:
# train test split
from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(df_X,
                                                    np.ravel(df_y),
                                                    test_size=0.3)

我们将训练**支持向量分类器**\(SVC\) 模型。正则化参数`C`和核系数`gamma`是 SVC 中最重要的两个超参数：

- 正则化参数`C`决定了正则化的强度。
- 核系数`gamma`控制核的宽度。SVC 默认使用**径向基函数 \(RBF\)**核（也称为**高斯核**）。

我们将在以下教程中调整这两个参数。

## 网格搜索

最优值`C`和`gamma`是比较难找得到的。最简单的解决方案是尝试一堆组合，看看哪种组合效果最好。这种创建参数“网格”并尝试所有可能组合的方法称为**网格搜索。**

![image.png](images/1.png)

这种方法非常常见，所以 Scikit-learn 在`GridSearchCV`中内置了这种功能。CV 代表交叉验证，这是另一种评估和改进机器学习模型的技术。

`GridSearchCV`需要一个描述准备尝试的参数和要训练的模型的字典。网格搜索的参数网格定义为字典，其中键是参数，值是要测试的一系列设置值。下面动手试试，首先定义候选参数` C``和``gamma `，如下所示：

In [5]:
param_grid = { 
  'C': [0.1, 1, 10, 100, 1000], 
  'gamma': [1, 0.1, 0.01, 0.001, 0.0001] 
}

接下来创建一个`GridSearchCV`对象，并使用训练数据进行训练模型。

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
svc1=SVC()
grid = GridSearchCV(svc1, param_grid, refit=True, verbose=3)

In [7]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1 ..................................................
[CV] ...................... C=0.1, gamma=1, score=0.600, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ...................... C=0.1, gamma=1, score=0.600, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ...................... C=0.1, gamma=1, score=0.600, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ...................... C=0.1, gamma=1, score=0.595, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ...................... C=0.1, gamma=1, score=0.608, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.600, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ..........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................... C=0.1, gamma=0.1, score=0.600, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.595, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.608, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.600, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.600, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.600, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.595, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] .

[CV] ................... C=10, gamma=0.001, score=0.950, total=   0.0s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.887, total=   0.0s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.886, total=   0.0s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.924, total=   0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.963, total=   0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.938, total=   0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.900, total=   0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:    2.6s finished


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
             verbose=3)

一旦训练完成后，我们可以通过`GridSearchCV`的`best_params_`属性查看搜索到的最佳参数，并使用`best_estimator_`属性查看最佳模型：

In [8]:
# 找到最好的参数
grid.best_params_

{'C': 1, 'gamma': 0.0001}

In [9]:
# 找到最好的模型
grid.best_estimator_

SVC(C=1, gamma=0.0001)

训练完成后，现在选择并采用该网格搜索到的最佳模型，并使用测试集进行预测并创建分类报告和混淆矩阵。

In [10]:
# 使用最好的估计器进行预测
grid_predictions = grid.predict(X_test)
# 混淆矩阵
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, grid_predictions))

[[ 42  11]
 [  1 117]]


In [11]:
# 分类模型报告
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.98      0.79      0.88        53
           1       0.91      0.99      0.95       118

    accuracy                           0.93       171
   macro avg       0.95      0.89      0.91       171
weighted avg       0.93      0.93      0.93       171



## 随机搜索

网格搜索尝试超参数的所有组合，因此增加了计算的时间复杂度，在数据量较大，或者模型较为复杂等等情况下，可能导致不可行的计算成本，这样网格搜索调参方法就不适用了。然而，**随机搜索**提供更便利的替代方案，该方法只测试你选择的超参数组成的元组，并且超参数值的选择是完全随机的，如下图所示。

![随机搜索尝试随机组合](images/2.png)

这种方法也很常见，所以 Scikit-learn 在`RandomizedSearchCV`中内置了这种功能。函数 API 与`GridSearchCV`类似。首先指定参数`C`和`gamma`以及参数值的候选样本的分布，如下所示:

In [12]:
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
from sklearn.utils.fixes import loguniform

# 指定采样的参数和分布
param_dist = {
  'C': stats.uniform(0.1, 1e4),
  'gamma': loguniform(1e-6, 1e+1),
}

接下来创建一个`RandomizedSearchCV`带参数`n_iter_search`的对象，并将使用训练数据来训练模型。

In [13]:
n_iter_search = 20
random_search = RandomizedSearchCV(SVC(),
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   refit=True,
                                   verbose=3)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] C=1774.343244541459, gamma=3.1892778886544583e-06 ...............
[CV]  C=1774.343244541459, gamma=3.1892778886544583e-06, score=0.988, total=   0.0s
[CV] C=1774.343244541459, gamma=3.1892778886544583e-06 ...............
[CV]  C=1774.343244541459, gamma=3.1892778886544583e-06, score=0.938, total=   0.0s
[CV] C=1774.343244541459, gamma=3.1892778886544583e-06 ...............
[CV]  C=1774.343244541459, gamma=3.1892778886544583e-06, score=0.938, total=   0.0s
[CV] C=1774.343244541459, gamma=3.1892778886544583e-06 ...............
[CV]  C=1774.343244541459, gamma=3.1892778886544583e-06, score=0.899, total=   0.0s
[CV] C=1774.343244541459, gamma=3.1892778886544583e-06 ...............
[CV]  C=1774.343244541459, gamma=3.1892778886544583e-06, score=0.962, total=   0.0s
[CV] C=1503.2524541228202, gamma=0.10464256812063606 .................
[CV]  C=1503.2524541228202, gamma=0.10464256812063606, score=0.600, total=   0.0s
[CV] C=150

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  C=1503.2524541228202, gamma=0.10464256812063606, score=0.595, total=   0.0s
[CV] C=1503.2524541228202, gamma=0.10464256812063606 .................
[CV]  C=1503.2524541228202, gamma=0.10464256812063606, score=0.608, total=   0.0s
[CV] C=1175.6858596772884, gamma=0.001591903045908838 ................
[CV]  C=1175.6858596772884, gamma=0.001591903045908838, score=0.938, total=   0.0s
[CV] C=1175.6858596772884, gamma=0.001591903045908838 ................
[CV]  C=1175.6858596772884, gamma=0.001591903045908838, score=0.925, total=   0.0s
[CV] C=1175.6858596772884, gamma=0.001591903045908838 ................
[CV]  C=1175.6858596772884, gamma=0.001591903045908838, score=0.887, total=   0.0s
[CV] C=1175.6858596772884, gamma=0.001591903045908838 ................
[CV]  C=1175.6858596772884, gamma=0.001591903045908838, score=0.835, total=   0.0s
[CV] C=1175.6858596772884, gamma=0.001591903045908838 ................
[CV]  C=1175.6858596772884, gamma=0.001591903045908838, score=0.924, total=   

[CV]  C=8646.69312648182, gamma=0.0514422048188468, score=0.600, total=   0.0s
[CV] C=8646.69312648182, gamma=0.0514422048188468 ....................
[CV]  C=8646.69312648182, gamma=0.0514422048188468, score=0.595, total=   0.0s
[CV] C=8646.69312648182, gamma=0.0514422048188468 ....................
[CV]  C=8646.69312648182, gamma=0.0514422048188468, score=0.608, total=   0.0s
[CV] C=298.3475123803619, gamma=0.00010372525056028988 ...............
[CV]  C=298.3475123803619, gamma=0.00010372525056028988, score=0.950, total=   0.0s
[CV] C=298.3475123803619, gamma=0.00010372525056028988 ...............
[CV]  C=298.3475123803619, gamma=0.00010372525056028988, score=0.938, total=   0.0s
[CV] C=298.3475123803619, gamma=0.00010372525056028988 ...............
[CV]  C=298.3475123803619, gamma=0.00010372525056028988, score=0.912, total=   0.0s
[CV] C=298.3475123803619, gamma=0.00010372525056028988 ...............
[CV]  C=298.3475123803619, gamma=0.00010372525056028988, score=0.911, total=   0.0s
[

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.9s finished


RandomizedSearchCV(estimator=SVC(), n_iter=20,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025DC039ED60>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025DC039E2B0>},
                   verbose=3)

同样，一旦训练完成后，我们可以通过`RandomizedSearchCV`的`best_params_`属性查看搜索到的最佳参数，并使用`best_estimator_`属性查看得到的最佳模型：

预测 RandomizedSearchCV 并创建报告。

最后，我们采用最终确定的最佳随机搜索模型，并使用测试集进行预测，并创建分类报告和混淆矩阵查看模型效果。

In [14]:
# 使用最好的估计器进行预测
random_predictions = random_search.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
# Confusion matrics
print(confusion_matrix(y_test, random_predictions))

[[ 45   8]
 [  2 116]]


In [15]:
# 分类评价报告
print(classification_report(y_test, random_predictions))

              precision    recall  f1-score   support

           0       0.96      0.85      0.90        53
           1       0.94      0.98      0.96       118

    accuracy                           0.94       171
   macro avg       0.95      0.92      0.93       171
weighted avg       0.94      0.94      0.94       171



## 贝叶斯搜索

贝叶斯搜索使用贝叶斯优化技术对搜索空间进行建模，以尽快获得优化的参数值。它使用搜索空间的结构来优化搜索时间。贝叶斯搜索方法使用过去的评估结果来采样最有可能提供更好结果的新候选参数（如下图所示）。

![贝叶斯搜索](images/3.png)

[Scikit-Optimize](undefined "undefined")库带有 BayesSearchCV 实现。

首先指定参数 C 和 gamma 以及参数值的候选样本的分布，如下所示:

In [16]:
# !pip install scikit-optimize

In [17]:
from skopt import BayesSearchCV
# 参数范围由下面的一个指定
from skopt.space import Real, Categorical, Integer

search_spaces = {
    'C': Real(0.1, 1e+4),
    'gamma': Real(1e-6, 1e+1, 'log-uniform'),
}

接下来创建一个 BayesSearchCV 带参数 n_iter_search 的对象，并将使用训练数据来训练模型。

In [18]:
n_iter_search = 20
bayes_search = BayesSearchCV(SVC(),
                             search_spaces,
                             n_iter=n_iter_search,
                             cv=5,
                             verbose=3)
bayes_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=4977.397744560771, gamma=0.023829160053700667 .................
[CV]  C=4977.397744560771, gamma=0.023829160053700667, score=0.600, total=   0.0s
[CV] C=4977.397744560771, gamma=0.023829160053700667 .................
[CV]  C=4977.397744560771, gamma=0.023829160053700667, score=0.600, total=   0.0s
[CV] C=4977.397744560771, gamma=0.023829160053700667 .................
[CV]  C=4977.397744560771, gamma=0.023829160053700667, score=0.600, total=   0.0s
[CV] C=4977.397744560771, gamma=0.023829160053700667 .................
[CV]  C=4977.397744560771, gamma=0.023829160053700667, score=0.595, total=   0.0s
[CV] C=4977.397744560771, gamma=0.023829160053700667 .................
[CV]  C=4977.397744560771, gamma=0.023829160053700667, score=0.608, total=   0.0s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=8758.852767384362, gamma=4.852350395777465 ....................
[CV]  C=8758.852767384362, gamma=4.85235039577

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  C=8758.852767384362, gamma=4.852350395777465, score=0.595, total=   0.0s
[CV] C=8758.852767384362, gamma=4.852350395777465 ....................
[CV]  C=8758.852767384362, gamma=4.852350395777465, score=0.608, total=   0.0s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=2910.2258495677825, gamma=0.9018344522099031 ..................
[CV]  C=2910.2258495677825, gamma=0.9018344522099031, score=0.600, total=   0.0s
[CV] C=2910.2258495677825, gamma=0.9018344522099031 ..................
[CV]  C=2910.2258495677825, gamma=0.9018344522099031, score=0.600, total=   0.0s
[CV] C=2910.2258495677825, gamma=0.9018344522099031 ..................
[CV]  C=2910.2258495677825, gamma=0.9018344522099031, score=0.600, total=   0.0s
[CV] C=2910.2258495677825, gamma=0.9018344522099031 ..................
[CV]  C=2910.2258495677825, gamma=0.9018344522099031, score=0.595, total=   0.0s
[CV] C=2910.2258495677825, gamma=0.9018344522099031 ..................
[CV]  C=2910.2258495677825, gamma

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] C=6201.076450375709, gamma=4.284329825969826e-06 ................
[CV]  C=6201.076450375709, gamma=4.284329825969826e-06, score=0.988, total=   0.0s
[CV] C=6201.076450375709, gamma=4.284329825969826e-06 ................
[CV]  C=6201.076450375709, gamma=4.284329825969826e-06, score=0.925, total=   0.0s
[CV] C=6201.076450375709, gamma=4.284329825969826e-06 ................
[CV]  C=6201.076450375709, gamma=4.284329825969826e-06, score=0.938, total=   0.0s
[CV] C=6201.076450375709, gamma=4.284329825969826e-06 ................
[CV]  C=6201.076450375709, gamma=4.284329825969826e-06, score=0.924, total=   0.0s
[CV] C=6201.076450375709, gamma=4.284329825969826e-06 ................
[CV]  C=6201.076450375709, gamma=4.284329825969826e-06, score=0.949, total=   0.0s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=8519.566314535297, gamma=5.566534140000436 ....................
[CV]  C=8519.566314535297, gamma=5.566534140000436, score=0.600, total=   0.0s
[CV] C=8519.56631453

...

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=9914.701087332833, gamma=0.00013060755725868254 ...............
[CV]  C=9914.701087332833, gamma=0.00013060755725868254, score=0.963, total=   0.0s
[CV] C=9914.701087332833, gamma=0.00013060755725868254 ...............
[CV]  C=9914.701087332833, gamma=0.00013060755725868254, score=0.912, total=   0.0s
[CV] C=9914.701087332833, gamma=0.00013060755725868254 ...............
[CV]  C=9914.701087332833, gamma=0.00013060755725868254, score=0.912, total=   0.0s
[CV] C=9914.701087332833, gamma=0.00013060755725868254 ...............
[CV]  C=9914.701087332833, gamma=0.00013060755725868254, score=0.899, total=   0.0s
[CV] C=9914.701087332833, gamma=0.00013060755725868254 ...............
[CV]  C=9914.701087332833, gamma=0.00013060755725868254, score=0.899, total=   0.0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=34.78128435599578, gamma=0.00046475689572079325 ...............
[CV]  C=34.78128435599578, gamma=0.00046475689572079325, score=0.963, total=   0.0s
[CV] C=34.78128435599578, gamma=0.00046475689572079325 ...............
[CV]  C=34.78128435599578, gamma=0.00046475689572079325, score=0.938, total=   0.0s
[CV] C=34.78128435599578, gamma=0.00046475689572079325 ...............
[CV]  C=34.78128435599578, gamma=0.00046475689572079325, score=0.875, total=   0.0s
[CV] C=34.78128435599578, gamma=0.00046475689572079325 ...............
[CV]  C=34.78128435599578, gamma=0.00046475689572079325, score=0.886, total=   0.0s
[CV] C=34.78128435599578, gamma=0.00046475689572079325 ...............
[CV]  C=34.78128435599578, gamma=0.00046475689572079325, score=0.924, total=   0.0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


BayesSearchCV(cv=5, estimator=SVC(), n_iter=20,
              search_spaces={'C': Real(low=0.1, high=10000.0, prior='uniform', transform='normalize'),
                             'gamma': Real(low=1e-06, high=10.0, prior='log-uniform', transform='normalize')},
              verbose=3)

同样，一旦训练完成后，我们可以通过检查发现的最佳参数`BayesSearchCV`的`best_params_`属性，并在最佳估计`best_estimator_`属性：

In [19]:
bayes_search.best_params_

OrderedDict([('C', 10000.0), ('gamma', 1e-06)])

In [20]:
bayes_search.best_estimator_

SVC(C=10000.0, gamma=1e-06)

最后，我们采用贝叶斯搜索模型并使用测试集创建一些预测，并为它们创建分类报告和混淆矩阵。

In [21]:
bayes_predictions = bayes_search.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
# 混淆矩阵
print(confusion_matrix(y_test, bayes_predictions))

[[ 46   7]
 [  1 117]]


In [22]:
# 分类评价报告
print(classification_report(y_test, bayes_predictions))

              precision    recall  f1-score   support

           0       0.98      0.87      0.92        53
           1       0.94      0.99      0.97       118

    accuracy                           0.95       171
   macro avg       0.96      0.93      0.94       171
weighted avg       0.95      0.95      0.95       171



## 写在最后


在本文中，我们介绍了 3 种最流行的超参数优化技术，这些技术用于获得最佳超参数集，从而训练稳健的机器学习模型。

一般来说，如果组合的数量足够有限，我们可以使用**网格搜索**技术。但是当组合数量增加时，我们应该尝试**随机搜索**或**贝叶斯搜索**，因为它们在计算上并不非常消耗资源。